<a href="https://colab.research.google.com/github/hwangdonggeun/project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install pykrx

import os
import pandas as pd
from pykrx import stock
from datetime import datetime, timedelta
import smtplib
from email.mime.text import MIMEText
from getpass import getpass

SENT_FILE = "sent_dates.txt"

def was_sent(ref_date):
    if not os.path.exists(SENT_FILE):
        return False
    with open(SENT_FILE, "r") as f:
        sent = {line.strip() for line in f}
    return ref_date in sent

def mark_sent(ref_date):
    with open(SENT_FILE, "a") as f:
        f.write(ref_date + "\n")

def load_returns(tickers, start_date, end_date):
    df = pd.DataFrame()
    for t in tickers:
        ohlcv = stock.get_market_ohlcv(start_date, end_date, t)
        ohlcv.columns = [
            col.strip() if isinstance(col, str) else col
            for col in ohlcv.columns
        ]
        if '등락률' in ohlcv.columns:
            ret = ohlcv['등락률']
        elif '종가' in ohlcv.columns:
            ret = ohlcv['종가'].pct_change()
        elif 'close' in ohlcv.columns:
            ret = ohlcv['close'].pct_change()
        elif len(ohlcv.columns) >= 4:
            ret = ohlcv.iloc[:, 3].pct_change()
        else:
            return pd.DataFrame()
        df[t] = ret.dropna()
    return df

def compute_thresholds(returns, quantile):
    return returns.quantile(quantile)

def detect_joint_extreme(today_ret, thresholds):
    return (today_ret < thresholds).all()

def get_last_trading_date(ref_yyyymmdd, ticker):
    d = datetime.strptime(ref_yyyymmdd, "%Y%m%d")
    for _ in range(7):
        ds = d.strftime("%Y%m%d")
        if not stock.get_market_ohlcv(ds, ds, ticker).empty:
            return ds
        d -= timedelta(days=1)
    return None

def send_email(subject, body, sender, pwd, receiver):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From']    = sender
    msg['To']      = receiver

    smtp_server = 'smtp.gmail.com'
    smtp_port   = 587
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(sender, pwd)
    server.send_message(msg)
    server.quit()

tickers   = input("모니터링할 종목코드(콤마 구분): ").split(',')
start_dt  = input("과거 시작일(YYYYMMDD): ")
end_dt    = input("과거 종료일(YYYYMMDD): ")
quantile  = float(input("위험 분위수(0~1): "))
sender    = input("발신 이메일(Gmail): ")
pwd       = getpass("앱 비밀번호(16자리): ")
receiver  = input("수신 이메일: ")

today     = datetime.now().strftime("%Y%m%d")

past       = load_returns(tickers, start_dt, end_dt)
thresholds = compute_thresholds(past, quantile)

ref_date = get_last_trading_date(today, tickers[0])
if ref_date != today:
    print(f"⚠️ 오늘 휴장, 최신 영업일: {ref_date}")

today_df  = load_returns(tickers, ref_date, ref_date)
today_ret = today_df.iloc[0]

if detect_joint_extreme(today_ret, thresholds):
    if was_sent(ref_date):
        print(f"⚠️ 이미 {ref_date}에 알림을 보냈습니다. 중복 발송 스킵")
    else:
        subj = f"[ALERT] Joint Extreme {ref_date}"
        body = (
            f"현재 수익률:\n{today_ret.to_string()}\n\n"
            f"기준치:\n{thresholds.to_string()}"
        )
        send_email(subj, body, sender, pwd, receiver)
        mark_sent(ref_date)
        print("✅ 메일 발송 완료")
else:
    print("✅ 위험 수준 아님")


모니터링할 종목코드(콤마 구분): 263750
과거 시작일(YYYYMMDD): 20250618
과거 종료일(YYYYMMDD): 20250619
위험 분위수(0~1): 1
발신 이메일(Gmail): donggeun21127@gmail.com
앱 비밀번호(16자리): ··········
수신 이메일: 2020131042@yonsei.ac.kr
⚠️ 오늘 휴장, 최신 영업일: 20250620
✅ 메일 발송 완료
